In [10]:
!pip install sodapy

In [15]:
from sodapy import Socrata

In [209]:
from numpy import array
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Dense
from keras.layers import Dropout
import pandas as pd
#from sodapy import Socrata



In [153]:
#Uncomment the below to open data source, download sensor data, and store it as a csv locally.

##Function to get Sensor count history data
def sensor_count():
 client = Socrata('data.melbourne.vic.gov.au', 'nlPM0PQJSjzCsbVqntjPvjB1f', None)
 sensor_data_id = "b2ak-trbp"
 results = client.get(sensor_data_id, limit=5000000)
 df = pd.DataFrame.from_records(results)
 df = df[['date_time', 'year', 'month', 'mdate', 'day', 'time', 'sensor_id', 'sensor_name', 'hourly_counts']]
 return df

sensor_history = sensor_count()

sensor_history.to_csv('sensor_history.csv', index=False)

In [167]:
sensor_history = pd.read_csv('sensor_history.csv')

In [171]:
single_sensor = sensor_history[sensor_history.sensor_id == 4]

In [172]:
single_sensor

,date_time,year,month,mdate,day,time,sensor_id,sensor_name,hourly_counts
26,2019-11-01T18:00:00.000,2019,November,1,Friday,18,4,Town Hall (West),2950
81,2019-11-01T19:00:00.000,2019,November,1,Friday,19,4,Town Hall (West),2345
137,2019-11-01T20:00:00.000,2019,November,1,Friday,20,4,Town Hall (West),2051
193,2019-11-01T21:00:00.000,2019,November,1,Friday,21,4,Town Hall (West),1715
248,2019-11-01T22:00:00.000,2019,November,1,Friday,22,4,Town Hall (West),1438
...,...,...,...,...,...,...,...,...,...
4231010,2022-04-30T19:00:00.000,2022,April,30,Saturday,19,4,Town Hall (West),1641
4231011,2022-04-30T20:00:00.000,2022,April,30,Saturday,20,4,Town Hall (West),1239
4231012,2022-04-30T21:00:00.000,2022,April,30,Saturday,21,4,Town Hall (West),1083
4231013,2022-04-30T22:00:00.000,2022,April,30,Saturday,22,4,Town Hall (West),883


In [411]:
sensor_seq = []
for i in range(0, len(single_sensor)):
    sensor_seq.append(single_sensor.hourly_counts.iloc[i])
    

In [412]:
# split a univariate sequence
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# choose a number of time steps
n_steps = 7
# split into samples
X, y = split_sequence(sensor_seq, n_steps)


In [413]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))


In [492]:
#Setting hyperparameters
units = 7
dropout_rate = 0.4
learning_rate = 0.0007
decay_rate = 0.97
momentum = 0.9
epochs = 11
batch_size = 16

#Define model
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
model.add(Dropout(0.3))
model.add(Dense(1))
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate) 

#Compile Model
model.compile(optimizer=opt, loss='mse', metrics = ['accuracy'])


In [493]:
#Fit model
model.fit(X, y, epochs=15, verbose=1)


Epoch 1/15
3368/3368 [==============================] - 35s 8ms/step - loss: 264735.4375 - accuracy: 3.6187e-04
Epoch 2/15
3368/3368 [==============================] - 29s 9ms/step - loss: 181119.7344 - accuracy: 3.5259e-04
Epoch 3/15
3368/3368 [==============================] - 35s 10ms/step - loss: 152851.3750 - accuracy: 3.8971e-04
Epoch 4/15
3368/3368 [==============================] - 40s 12ms/step - loss: 135451.0781 - accuracy: 3.8971e-04
Epoch 5/15
3368/3368 [==============================] - 25s 8ms/step - loss: 128822.5781 - accuracy: 3.7115e-04
Epoch 6/15
3368/3368 [==============================] - 25s 7ms/step - loss: 119844.8438 - accuracy: 3.9899e-04
Epoch 7/15
3368/3368 [==============================] - 32s 9ms/step - loss: 123717.7422 - accuracy: 3.6187e-04: 0s - loss: 123792.7188
Epoch 8/15
3368/3368 [==============================] - 38s 11ms/step - loss: 122927.5391 - accuracy: 3.8971e-04
Epoch 9/15
3368/3368 [==============================] - 31s 9ms/step - loss: 

In [494]:
# demonstrate prediction
x_input = array(sensor_seq[1:8])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print("predicted next value: ", int(yhat))
print("actual next value: ", sensor_seq[8])




predicted next value:  287
actual next value:  205


In [495]:
# demonstrate prediction
x_input = array(sensor_seq[21:28])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print("predicted next value: ", int(yhat))
print("actual next value: ", sensor_seq[28])



predicted next value:  1075
actual next value:  1228
